# Logistic Regression Model

_Kevin Siswandi_  
**Fundamentals of Machine Learning**  
June 2020

Assume a binary classification problem with $y \in \{1, 0\}$. The output of a logistic regression classifier is

$$ f(x; w) = \sigma(w^T x) $$

where $\sigma(...)$ is the sigmoid function. This is handy because it can be interpreted as probability of having y = 1.

## Sigmoid function
 
The sigmoid or logistic function is given by

$$ \sigma(z) = \frac{1}{1+e^{-z}} $$

## Nonlinear decision boundary

Logistic regression can learn nonlinear decision boundary by introducing the right features -- e.g. polynomial or higher order features.

## Training

The optimization objective is

$$ \arg \min_w \sum_i - y_i \log(f(x_i, w)) - (1-y_i) \log(1-f(x_i, w))$$

We can't use the squared error cost function because it is nonconvex with multiple local minima. This cross-entropy cost function captures the intuition that if f = 0 when y = 1, the learning algorithm is penalized by a high cost. Moreover, it is convex and hence local minima free. The cross-entropy cost function can be derived using the principle of maximum-likelihood estimation.

To minimize the cost function, we use gradient descent. The GD update is

$$ w := w - \alpha \sum_i (f(x_i, w) - y_i) x_i $$ 

Note that feature scaling helps with gradient descent to converge faster.